In [1]:
import pandas as pd
import numpy as np
import json
import datetime 

In [300]:
sourceCheckins = pd.read_csv('data/July/server/checkins.tsv', sep='\t')
checkins = pd.read_csv('data/July/server/checkinsHistory.tsv', sep='\t')
checkins1 = pd.read_csv('data/July/server/extendedCheckinsHistoryJune.tsv', sep='\t')
checkpoints = pd.read_csv('data/July/server/checkPoints.tsv', sep='\t')

In [308]:
len(np.unique(allCheckins.userId.tolist()))

1537

In [306]:
len(allCheckins)

175889

In [302]:
allCheckins = allCheckins.drop_duplicates(subset = ['checkinId'])

In [301]:
allCheckins = pd.concat([checkins1, checkins])

In [3]:
allCheckins = pd.read_csv('data/July/server/currentExtendedCheckinsHistory.tsv', sep='\t')

In [309]:
allCheckins.to_csv('data/July/server/currentExtendedCheckinsHistory.tsv', sep='\t', encoding='utf-8')

In [305]:
allCheckins.drop('Unnamed: 0.1', axis=1, inplace=True)

In [310]:
sortedVenues = []
sortedVenuesIds = []
for venueId, group in allCheckins.groupby('venueId').agg('size').order(ascending = False).iteritems():
    sortedVenues.append({'venueId':venueId,
                        'group':group})
    sortedVenuesIds.append(venueId)

In [115]:
#'World Class Lite in top ten!, 520f117111d2a782f1211c3b'

In [311]:
groupedUsers = allCheckins.groupby('userId')
userVenues = []
for index, group in enumerate(groupedUsers):
        userVenues.append(group[1]['venueId'].tolist())

In [312]:
len(sortedVenues)

44700

In [18]:
top50VenuesCheckins = allCheckins[allCheckins.venueId.isin(sortedVenuesIds[:50])]

In [23]:
for venueId in sortedVenuesIds[:50]:
    print venueNameFromId(venueId)

ЦПКиО им. Горького / Gorky Park
Москва / Moscow
Парк искусств «Музеон» / Muzeon Park
ВДНХ (Выставка достижений народного хозяйства)
Россия / Russia
Международный аэропорт Домодедово / Domodedovo International Airport (DME)
ПКиО «Сокольники»
Красная площадь / Red Square
ТРЦ «Европейский»
Арбат / Arbat Street
Международный аэропорт Шереметьево / Sheremetyevo International Airport (SVO)
ТРК «Атриум» / Atrium Mall
ТЦ «Метрополис»
Афимолл Сити / Afimall City
Физика
Центральный детский магазин
Международный аэропорт Внуково / Vnukovo International Airport (VKO)
Ленинградский вокзал / Leningradsky Railway Terminal
Музей-заповедник «Коломенское»
Нескучный сад
ТЦ «Охотный ряд»
ТРЦ «Авиапарк»
Планета Фитнес
World Class Lite
ГУМ / GUM
Воробьёвская набережная
Патриаршие пруды / Patriarshiye Ponds
ПКиО «Измайловский»
Московский вокзал / Moskovsky Railway Station
Tsvetnoy Central Market
Парк Победы (Поклонная гора) / Victory Park
Фитнес-мания
Казанский вокзал / Kazansky Rail Terminal
ТРЦ «Золотой Ва

In [ ]:
from pymining import itemmining
relim_input = itemmining.get_relim_input(userVenues)
report = itemmining.relim(relim_input, min_support=2)
report

In [313]:
def load_dataset():
    "Load the sample dataset."
    return userVenues
 
 
def createC1(dataset):
    "Create a list of candidate item sets of size one."
    c1 = []
    for transaction in dataset:
        for item in transaction:
            if not [item] in c1:
                c1.append([item])
    c1.sort()
    #frozenset because it will be a ket of a dictionary.
    return map(frozenset, c1)
 
 
def scanD(dataset, candidates, min_support):
    "Returns all candidates that meets a minimum support level"
    sscnt = {}
    for tid in dataset:
        for can in candidates:
            if can.issubset(tid):
                sscnt.setdefault(can, 0)
                sscnt[can] += 1
 
    num_items = float(len(dataset))
    retlist = []
    support_data = {}
    for key in sscnt:
        support = sscnt[key] / num_items
        if support >= min_support:
            retlist.insert(0, key)
        support_data[key] = support
    return retlist, support_data
 
 
def aprioriGen(freq_sets, k):
    "Generate the joint transactions from candidate sets"
    retList = []
    lenLk = len(freq_sets)
    for i in range(lenLk):
        for j in range(i + 1, lenLk):
            L1 = list(freq_sets[i])[:k - 2]
            L2 = list(freq_sets[j])[:k - 2]
            L1.sort()
            L2.sort()
            if L1 == L2:
                retList.append(freq_sets[i] | freq_sets[j])
    return retList
 
 
def apriori(dataset, minsupport=0.5):
    "Generate a list of candidate item sets"
    C1 = createC1(dataset)
    D = map(set, dataset)
    L1, support_data = scanD(D, C1, minsupport)
    L = [L1]
    k = 2
    while (len(L[k - 2]) > 0):
        Ck = aprioriGen(L[k - 2], k)
        Lk, supK = scanD(D, Ck, minsupport)
        support_data.update(supK)
        L.append(Lk)
        k += 1
 
    return L, support_data

In [ ]:
C1 = createC1(userVenues)

In [252]:
D = map(set, userVenues)

In [253]:
L1, support_data = scanD(D, C1, 0.01)

In [315]:
L, supportData = apriori(userVenues, minsupport=0.01)

In [316]:
frequentVenues = []
for itemset, support in zip(L, supportData):
    if len(itemset)>5:
        print support
        frequentVenueSet = []
        for item in itemset:
            frequentVenueSet.append(next(enumerate(item))[1])
        frequentVenues.append(frequentVenueSet)

frozenset(['4be6909fd4f7c9b6b2e02620', '4ba6146ef964a5202d3239e3'])
frozenset(['4bd855b5f645c9b6f5a1a7e0', '4bae5064f964a52077a13be3'])
frozenset(['4bdd5f27f219c9b6bb751210', '5526587c498e151e3d6ab2f4', '523c543311d2343c967a796e'])
frozenset(['534f9f0b498ea76789184be0', '515dcc1ae4b0b437d6ad9c66'])
frozenset(['534a9d1e498e5b2d5c3e5125'])
frozenset(['4c0b65a2bbc676b003864bd5', '4ba91963f964a520480b3ae3'])


In [40]:
len(userVenues)*0.0015455950541

1.0000000000027

In [319]:
for venueId in frequentVenues[0][:20]:
    print venueNameFromId(venueId)

Афимолл Сити / Afimall City
Гин-но Таки
McDonald's
Cuba Libre
Hard Rock Cafe Moscow
Conversation Cafe
Bar BQ Cafe
Страна которой нет
Каро Vegas 22
Площадь Киевского Вокзала
Счастье
Площадь Восстания
Starbucks
Аэроэкспресс - Терминал на Павелецком вокзале
Osteria della Piazza Bianca
ТРK «Вегас» (Крокус Сити)
Декатлон
ТК «Горбушка»
Теплый Стан
Площадь Революции


In [318]:
def generateRules(L, support_data, min_confidence=0.7):
    """Create the association rules
    L: list of frequent item sets
    support_data: support data for those itemsets
    min_confidence: minimum confidence threshold
    """
    rules = []
    for i in range(1, len(L)):
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            #print "freqSet", freqSet, 'H1', H1
            if (i > 1):
                rules_from_conseq(freqSet, H1, support_data, rules, min_confidence)
            else:
                calc_confidence(freqSet, H1, support_data, rules, min_confidence)
    return rules
 
 
def calc_confidence(freqSet, H, support_data, rules, min_confidence=0.7):
    "Evaluate the rule generated"
    pruned_H = []
    for conseq in H:
        conf = support_data[freqSet] / support_data[freqSet - conseq]
        if conf >= min_confidence:
            print freqSet - conseq, '--->', conseq, 'conf:', conf
            rules.append((freqSet - conseq, conseq, conf))
            pruned_H.append(conseq)
    return pruned_H
 
 
def rules_from_conseq(freqSet, H, support_data, rules, min_confidence=0.7):
    "Generate a set of candidate rules"
    m = len(H[0])
    if (len(freqSet) > (m + 1)):
        Hmp1 = aprioriGen(H, m + 1)
        Hmp1 = calc_confidence(freqSet, Hmp1,  support_data, rules, min_confidence)
        if len(Hmp1) > 1:
            rules_from_conseq(freqSet, Hmp1, support_data, rules, min_confidence)

In [ ]:
rules = generateRules(L, supportData, min_confidence=0.6)

# Helper Functions

In [37]:
venuesDict = {}
def venueNameFromId(venueId):
    if venueId not in venuesDict.keys():
        venuesDict[venueId] = allCheckins[allCheckins.venueId == venueId].iloc[0]['name']
    return venuesDict[venueId]

In [38]:
venuesCatDict = {}
def venueCategoryFromId(venueId):
    if venueId not in venuesDict.keys():
        venuesDict[venueId] = allCheckins[allCheckins.venueId == venueId].iloc[0]['categoryName']
    return venuesDict[venueId]

In [39]:
categoriesDict = {}
def categoryNameFromId(categoryId):
    if categoryId not in categoriesDict.keys():
        categoriesDict[categoryId] = allCheckins[allCheckins.categoryId == categoryId].iloc[0]['categoryName']
    return categoriesDict[categoryId]

In [40]:
def countUsersAtVenue(venuedId):
    return len(np.unique(allCheckins[allCheckins.venueId == venueId]['userId'].tolist()))

In [321]:
for rule in rules:
    simpleRule = 1
    if len(rule[0]) < 2:
        continue
        
    for venueId in enumerate(rule[0]):
        if venueId[1] not in sortedVenuesIds[:20]:
            simpleRule = 0
            
    if simpleRule:
        continue
    
    simpleRule = 1
    
    for venueId in enumerate(rule[1]):
        if venueId[1] not in sortedVenuesIds[:20]:
            simpleRule = 0
            
    if simpleRule:
        continue
    
    for venueId in enumerate(rule[0]):
        print venueNameFromId(venueId[1])
        
    print '--->'
    for venueId in enumerate(rule[1]):
        print venueNameFromId(venueId[1])
    print 'conf:', rule[2]
    print ''

ТРЦ «Галерея» / Galeria Shopping Mall
ВДНХ (Выставка достижений народного хозяйства)
--->
Ленинградский вокзал / Leningradsky Railway Terminal
Московский вокзал / Moskovsky Railway Station
conf: 0.615384615385

Невский проспект / Nevsky Prospect
Московский вокзал / Moskovsky Railway Station
--->
Ленинградский вокзал / Leningradsky Railway Terminal
Дворцовая площадь / Palace Square
conf: 0.607142857143

Парк искусств «Музеон» / Muzeon Park
Каро 17 SKY
--->
ЦПКиО им. Горького / Gorky Park
ТРЦ «Авиапарк»
conf: 0.64

Государственный Эрмитаж / Hermitage Museum
Ленинградский вокзал / Leningradsky Railway Terminal
--->
ЦПКиО им. Горького / Gorky Park
Московский вокзал / Moskovsky Railway Station
conf: 0.64

ВДНХ (Выставка достижений народного хозяйства)
Дворцовая площадь / Palace Square
--->
Ленинградский вокзал / Leningradsky Railway Terminal
Московский вокзал / Moskovsky Railway Station
conf: 0.606060606061

Ленинградский вокзал / Leningradsky Railway Terminal
Невский проспект / Nevsky Pros

In [ ]:
#Using code for rules from here: http://aimotion.blogspot.ru/2013/01/machine-learning-and-data-mining.html

In [268]:
multiUserSortedVenues = filter(lambda x: x['group'] > 1, sortedVenues)

In [327]:
userVenueGroups = []
for venueId, group in allCheckins.groupby('venueId'):
    if len(group.groupby('userId'))>2:
        userVenueGroups.append(venueId)

In [323]:
userVenueCategories = []
for categoryId, group in allCheckins.groupby('categoryId'):
    if len(group.groupby('userId'))>1:
        userVenueCategories.append(categoryId)

In [328]:
#Make lists of users check-in venues
multiUserCheckins = allCheckins[allCheckins.venueId.isin(userVenueGroups)]
groupedUsers = multiUserCheckins.groupby('userId')
userVenuesList = []
userIds = []
for index, group in enumerate(groupedUsers):
        userVenuesList.append(group[1]['venueId'].tolist())
        userIds.append(group[0])

In [325]:
#Make lists of users check-in catrgories
fineCheckins = allCheckins[allCheckins.categoryId.isin(userVenueCategories)]
groupedUsers = fineCheckins.groupby('userId')
userCategoryList = []
userIdsForCat = []
for index, group in enumerate(groupedUsers):
        userCategoryList.append(group[1]['categoryId'].tolist())
        userIdsForCat.append(group[0])

In [329]:
len(userVenueGroups)

5977

# K-means, spectral

In [24]:
#Let's build some TF-ID vectors
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster.bicluster import SpectralCoclustering
from sklearn.cluster import MiniBatchKMeans

In [330]:
venuesByUser = []
for userList in userVenuesList:
    venuesByUser.append(' '.join(userList))
    
categoriesByUser = []
for userList in userCategoryList:
    try:
        categoriesByUser.append(' '.join(userList))
    except:
        print userList

tfidf_vectorizer = TfidfVectorizer(vocabulary = userVenueGroups)
tfidf_matrix_train = tfidf_vectorizer.fit_transform(venuesByUser)

In [331]:
from time import time
cocluster = SpectralCoclustering(n_clusters=20,
                                 svd_method='arpack', random_state=0)
kmeans = MiniBatchKMeans(n_clusters=20,
                         random_state=0)

print("Coclustering...")
start_time = time()
cocluster.fit(tfidf_matrix_train)
y_cocluster = cocluster.row_labels_
print("Done in {:.2f}s.".format(
    time() - start_time))

print("MiniBatchKMeans...")
start_time = time()
y_kmeans = kmeans.fit_predict(tfidf_matrix_train)
print("Done in {:.2f}s.".format(
    time() - start_time))

feature_names = tfidf_vectorizer.get_feature_names()

Coclustering...
Done in 0.33s.
MiniBatchKMeans...
Done in 0.10s.


In [131]:
clustersSpectral = [[] for i in range(20)]
for index, clusterId in enumerate(y_cocluster):
    clustersSpectral[clusterId].append(userIds[index])

In [154]:
np.std([len(cluster) for cluster in  clusters])

149.97869848748525

In [67]:
clusters = x = [[] for i in range(20)]
for index, clustedId in enumerate(y_kmeans):
    clusters[clustedId].append(userIds[index])

# LDA

In [332]:
import sklearn
from sklearn.feature_extraction.text import CountVectorizer 

cv = sklearn.feature_extraction.text.CountVectorizer()
mat = cv.fit_transform(venuesByUser).toarray()
print('Done Document matrix')

Done Document matrix


In [29]:
cvCategory = sklearn.feature_extraction.text.CountVectorizer()
matCategory = cvCategory.fit_transform(categoriesByUser).toarray()
print('Done Document matrix')

Done Document matrix


In [30]:
len(userVenueCategories)

520

In [333]:
venuesVocab = cv.get_feature_names()
print len(venuesVocab)

5977


In [32]:
categoriesVocab = cvCategory.get_feature_names()
print len(categoriesVocab)

520


In [362]:
import lda
model20 = lda.LDA(n_topics=20, n_iter=2000, random_state=1,alpha = 0.001)
model20.fit(mat)  # model.fit_transform(X) is also available
topic_word = model20.topic_word_  # model.components_ also works
print('Done LDA fitting')

Done LDA fitting


In [35]:
import lda
model20Cat = lda.LDA(n_topics=20, n_iter=2000, random_state=1,alpha = 0.001)
model20Cat.fit(matCategory)  # model.fit_transform(X) is also available
topic_cats = model20Cat.topic_word_  # model.components_ also works
print('Done LDA fitting')

Done LDA fitting


In [363]:
n_top_venues = 10
for i, venues_dist in enumerate(topic_word):
    indicies = np.argsort(venues_dist)[::-1]
    topic_venues = np.array(venuesVocab)[indicies]
    topic_probs = venues_dist[indicies]
    #print np.sort(venues_dist)[:-n_top_venues:-1]
    print('Cluster {}:'.format(i+1))
    for venueId, topicProb in zip(topic_venues, topic_probs):
        if topicProb>=0.01:
            print("{}, {} users {:.2f}%").format(venueNameFromId(venueId),countUsersAtVenue(venueId),topicProb*100)
        else:
            break
    print ''

Cluster 1:
ЦПКиО им. Горького / Gorky Park, 815 users 3.05%
Парк искусств «Музеон» / Muzeon Park, 470 users 1.79%
Международный аэропорт Домодедово / Domodedovo International Airport (DME), 354 users 1.34%
ПКиО «Сокольники», 305 users 1.31%
ВДНХ (Выставка достижений народного хозяйства), 441 users 1.08%

Cluster 2:
Москва / Moscow, 297 users 14.42%
Россия / Russia, 148 users 3.82%
World Class Lite, 12 users 3.31%
Gold's Gym, 11 users 2.71%
World Class Lite, 13 users 2.55%
Западный административный округ, 69 users 1.69%
ТРЦ «Калейдоскоп», 62 users 1.43%
ЦПКиО им. Горького / Gorky Park, 815 users 1.34%
Парк «Северное Тушино», 63 users 1.21%
Северный административный округ, 43 users 1.03%

Cluster 3:
Фитнес-мания, 21 users 2.16%
Международный аэропорт Шереметьево / Sheremetyevo International Airport (SVO), 228 users 2.13%
Яндекс / Yandex HQ, 26 users 1.37%
Международный аэропорт Домодедово / Domodedovo International Airport (DME), 354 users 1.27%
Game Insight, 3 users 1.13%
Alex Fitness, 

In [359]:
def exportLDAClustersToCSV(venues_clusters, allCheckins):
    LDAclustersDF = pd.DataFrame()
    for clustedId, venues_dist in enumerate(venues_clusters):
        indicies = np.argsort(venues_dist)[::-1]
        
        topic_venues = np.array(venuesVocab)[indicies]
        topic_probs = venues_dist[indicies]

        topVenuesIds =  [venueId for venueId, topicProb in zip(topic_venues, topic_probs) if topicProb>=0.01]
        clusterCheckins = allCheckins[allCheckins.venueId.isin(topVenuesIds)]
        clusterCheckins["clusterId"] = clustedId+1
        LDAclustersDF = LDAclustersDF.append(clusterCheckins)
    return LDAclustersDF

In [364]:
LDAclustersDF = exportLDAClustersToCSV(topic_word, allCheckins)

/usr/local/lib/python2.7/site-packages/IPython/kernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [365]:
LDAclustersDF.to_csv('data/July/LDAClusteredCheckins.tsv', sep="\t", encoding="utf-8")

In [367]:
len(LDAclustersDF)

71528

In [112]:
clustersLDA = [[] for i in range(20)]
for index, userTopics in enumerate(model20.ndz_):
    clustersLDA[np.argmax(userTopics)].append(userIds[index])

In [113]:
#Match the clusters by min Jaccard Distance
import distance
LDAClustersMap = []
LDAClustersMapIndexes = []
# clustersLDACopy = clustersLDA
distancesMatrix = []
for clusterK in clusters:
    distances = [distance.jaccard(clusterLDA,clusterK) for clusterLDA in clustersLDA]
    distancesMatrix.append(distances)
    index = np.argmin(distances)
    LDAClustersMap.append(clustersLDA[index])
#     clustersLDACopy.remove(clustersLDA[index])

In [156]:
clustersSpectral1 = [[0] if len(cluster) == 0 else cluster for cluster in clusterSpectreal]

In [ ]:
distancesMatrixKSpect = []
for clusterK in clusters:
    distances = [distance.jaccard(clusterSpectreal,clusterK) for clusterSpectreal in clustersSpectral]
    distancesMatrixKSpect.append(distances)

In [114]:
#Average Jaccard Distance between clusters
print np.mean([distance.jaccard(clusterK, LDAClustersMap[index]) for index, clusterK in enumerate(clusters) if len(clusterK)])

0.891365675713


In [124]:
#Average Jaccard Distance between clusters
print np.mean(np.min(distancesMatrix, axis=1))

0.923955972999


In [299]:
n_top_venues = 10
for i, venues_dist in enumerate(topic_cats):
    indicies = np.argsort(venues_dist)[:-n_top_venues:-1]
    topic_venues = np.array(categoriesVocab)[indicies]
    topic_probs = venues_dist[indicies]
    print('Cluster {}:'.format(i+1))
    for categoryId, topicProb in zip(topic_venues, topic_probs):
        if topicProb>=0.01:
            print("{}, {:.2f}%").format(categoryNameFromId(categoryId),topicProb*100)
        else:
            break
    print ''

Cluster 1:
Airport, 8.23%
Hotel, 5.28%
Park, 4.16%
Café, 3.24%
Restaurant, 2.80%
City, 2.75%
Mall, 2.70%
Coffee Shop, 2.64%
Italian Restaurant, 1.99%

Cluster 2:
Park, 6.61%
Café, 4.62%
Coffee Shop, 4.17%
Airport, 3.67%
Mall, 3.58%
Hotel, 2.84%
Restaurant, 2.43%
Italian Restaurant, 2.05%
Nightclub, 2.02%

Cluster 3:
Park, 8.92%
Train Station, 3.47%
Home (private), 3.37%
Coffee Shop, 2.76%
Café, 2.70%
Concert Hall, 2.58%
Mall, 2.20%
Art Museum, 2.07%
Event Space, 1.79%

Cluster 4:
Housing Development, 11.51%
Building, 8.08%
Factory, 7.28%
General Entertainment, 6.93%
Tech Startup, 4.61%
Nightclub, 4.05%
Cocktail Bar, 3.76%
Gas Station / Garage, 2.88%
Café, 2.79%

Cluster 5:
Home (private), 23.30%
College Academic Building, 7.54%
Residential Building (Apartment / Condo), 6.79%
College Residence Hall, 6.25%
University, 6.02%
Metro Station, 4.20%
Plaza, 3.53%
Mall, 3.47%
Fast Food Restaurant, 3.17%

Cluster 6:
Residential Building (Apartment / Condo), 44.91%
Park, 5.73%
Mall, 3.25%
Neighbo

In [228]:
print len(set(multiUserCheckins[multiUserCheckins.userId == clusters[10][1]]['venueId'].tolist()).intersection(multiUserCheckins[multiUserCheckins.userId == clusters[10][2]]['venueId'].tolist()))
print len(set(multiUserCheckins[multiUserCheckins.userId == clusters[10][1]]['venueId'].tolist()).union(multiUserCheckins[multiUserCheckins.userId == clusters[10][2]]['venueId'].tolist()))

4
140
